In [31]:
import os
import librosa
import numpy as np
import pandas as pd

class FeatureExtractor:
    def __init__(self, root_dir, target_folders, labels, n_mfcc=40, n_mel=128, n_contrast=8):
        self.root_dir = root_dir
        self.target_folders = target_folders
        self.labels = labels
        self.n_mfcc = n_mfcc
        self.n_mel = n_mel
        self.n_contrast = n_contrast

    def load_data(self, file_path):
        data, sample_rate = librosa.load(file_path, sr=None)
        return data, sample_rate

    def extract_features(self, data, sample_rate):
        # 다양한 특성 추출
        mfccs = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=self.n_mfcc)
        mel = librosa.feature.melspectrogram(y=data, sr=sample_rate, n_mels=self.n_mel)
        contrast = librosa.feature.spectral_contrast(y=data, sr=sample_rate, n_bands=self.n_contrast-1) # contrast 특성은 대역 수보다 1 적은 값을 사용

        # 모든 특성을 하나의 벡터로 결합
        features = np.hstack([
            np.mean(mfccs, axis=1),
            np.mean(mel, axis=1),
            np.mean(contrast, axis=1),
        ])

        return features

    def process_files_in_folder(self, folder_name, label):
        folder_path = os.path.join(self.root_dir, folder_name)
        feature_list = []
        for file in os.listdir(folder_path):
            if file.endswith('.ogg'):
                file_path = os.path.join(folder_path, file)
                data, sample_rate = self.load_data(file_path)

                # 특성 추출
                features = self.extract_features(data, sample_rate)
                features = np.append(features, label)  # 라벨 추가
                feature_list.append(features)
                print(f'Processed {file_path}')

        return feature_list

    def process_target_folders(self):
        all_features = []
        for folder, label in zip(self.target_folders, self.labels):
            features = self.process_files_in_folder(folder, label)
            all_features.extend(features)

        # 데이터프레임 생성
        column_names = (
                [f'mfcc_{i}' for i in range(self.n_mfcc)] + 
                [f'mel_{i}' for i in range(self.n_mel)] + 
                [f'contrast_{i}' for i in range(self.n_contrast)] + ['label']
        )
        df = pd.DataFrame(all_features, columns=column_names)

        # CSV 파일로 저장
        df.to_csv('bird_sounds_features.csv', index=False)
        print('Features saved to bird_sounds_features.csv')

root_dir = './data/train_audio'
target_folders = ['abethr1', 'afbfly1']
labels = [0, 1]

feature_extractor = FeatureExtractor(root_dir, target_folders, labels)
feature_extractor.process_target_folders()

Processed ./data/train_audio/abethr1/XC363504.ogg
Processed ./data/train_audio/abethr1/XC639039.ogg
Processed ./data/train_audio/abethr1/XC531557.ogg
Processed ./data/train_audio/abethr1/XC363502.ogg
Processed ./data/train_audio/abethr1/XC363503.ogg
Processed ./data/train_audio/abethr1/XC363501.ogg
Processed ./data/train_audio/abethr1/XC585802.ogg
Processed ./data/train_audio/abethr1/XC432639.ogg
Processed ./data/train_audio/abethr1/XC756300.ogg
Processed ./data/train_audio/abethr1/XC606253.ogg
Processed ./data/train_audio/abethr1/XC379322.ogg
Processed ./data/train_audio/abethr1/XC128013.ogg
Processed ./data/train_audio/abethr1/XC467121.ogg
Processed ./data/train_audio/abethr1/XC467122.ogg
Processed ./data/train_audio/abethr1/XC616997.ogg
Processed ./data/train_audio/afbfly1/XC316010.ogg
Processed ./data/train_audio/afbfly1/XC206826.ogg
Processed ./data/train_audio/afbfly1/XC200995.ogg
Processed ./data/train_audio/afbfly1/XC718068.ogg
Processed ./data/train_audio/afbfly1/XC301282.ogg


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# 데이터셋 로드
data = pd.read_csv('bird_sounds_features.csv')

# 특징과 레이블 분리
X = data.drop('label', axis=1)
y = data['label']

# 데이터셋을 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=30)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # 훈련 세트 표준화
X_test = scaler.transform(X_test)  # 테스트 세트 표준화

# 분류기 정의
knn = KNeighborsClassifier(n_neighbors=3)

# 분류기 학습
knn.fit(X_train, y_train)

# 예측 수행
knn_predictions = knn.predict(X_test)

# 분류기 평가
print("KNN Classifier Report:")
print(classification_report(y_test, knn_predictions))
print("Accuracy:", accuracy_score(y_test, knn_predictions))

KNN Classifier Report:
              precision    recall  f1-score   support

         0.0       1.00      0.75      0.86         4
         1.0       0.86      1.00      0.92         6

    accuracy                           0.90        10
   macro avg       0.93      0.88      0.89        10
weighted avg       0.91      0.90      0.90        10

Accuracy: 0.9
